# 4.3. Aktives Reinforcement Learning
>## <ins>Table of contents</ins> <a name="up"></a>[<sup>[1]</sup>](#cite_note-1)
>* [**4.3.1. Exploration vs. Exploitation**](#4_3_1)
>* [**4.3.2. ε-greedy Learning**](#4_3_2)
>* [**4.3.3. Q-Learning**](#4_3_3)
>* [**4.3.4. Unterschiede zwischen ε-greedy Learning und Q-Learning**](#4_3_4)
>
>## <ins>Beispiele</ins>
>* [**Beispiel 1**: Das k-armige Banditenproblem](#b1)
>* [**Beispiel 2**: ε-greedy Learning des Staubsaugerproblems$](#b2)
>* [**Beispiel 3**: Q-Learning des Staubsaugerproblems](#b3)

##### **Problemstellung**:

Nachdem wir uns in den [Unterkapiteln 4.1](http://localhost:8888/lab/tree/KE4_Reinforcement_Learning/4.1.-Markov-Entscheidungsprozess.ipynb) und [4.2](http://localhost:8888/lab/tree/KE4_Reinforcement_Learning/4.2.-Passives-Reinforcement-Learning.ipynb) mit grundlegenden Fragestellungen und Teilproblemen des Reinforcement Learnings beschäftigt haben, kommen wir nun zu der eigentlichen Herausforderung: nämlich <ins>des Lernens der optimalen Strategie in einer nicht-deterministischen und unbekannten Umgebung</ins>.-

## 4.3.1. Exploration vs. Exploitation <a name="4_3_1"></a>
---

Für die optimale Strategie $\pi^*$ gilt (siehe auch Gleichung (1) aus Unterkapitel 4.1):

[$$\pi^*(s) = \arg\max_{a \in A} \sum_{s' \in S} P(s,a,s') \left( R(s,a,s') + \gamma U^\gamma_D(s') \right) \tag{1}$$](http://localhost:8888/lab/tree/KE4_Reinforcement_Learning/4.1.-Markov-Entscheidungsprozess.ipynb)  <a name="g1"></a>

Mit anderen Worten:
Wenn wir korrekte und vollständige Informationen zu den Nutzen aller Zustände ($U^γ_D$), zu den Belohnungen in den Zuständen ($R$) und den Zustandsübergangswahrscheinlichkeiten ($P$) haben, sollten wir in jedem Zustand die Aktion auswählen, die den erwarteten Nutzen maximiert.
- In [Unterkapitel 4.2](http://localhost:8888/lab/tree/KE4_Reinforcement_Learning/4.2.-Passives-Reinforcement-Learning.ipynb) haben wir Methoden behandelt, die zu einer gegebenen Strategie $π$ die Nutzen der Zustände $U^γ_D(s | π)$ berechnen.
- Außerdem wissen wir, dass für die optimale Strategie gilt $U^γ_D(s) = U^γ_D(s | π*)$.
- Bei dem Ansatz der adaptiven dynamischen Programmierung (siehe Abschnitt 4.2.1) konnten wir auch die Werte $P(s,a,s′)$ und $R(s,a,s′)$ approximieren.

Auf den ersten Blick scheint es, dass wir unter der Abschätzung von $U^γ_D(s)$ durch $U^γ_D(s | π)$ alle notwendigen Komponenten haben, um durch Gleichung (1) die optimale Strategie zu ermitteln. Allerdings gibt es hierbei zwei grundlegende Probleme.

Allerdings gibt es hierbei **zwei grundlegende Probleme**:

1. Mit der adaptiven dynamischen Programmierung können wir nur die Werte $P(s,a,s′)$ und $R(s,a,s′)$ für $a = π(s)$ approximieren. Da wir eine feste Strategie angenommen haben, hat unser Agent keine Erfahrung, was andere Aktionen im Zustand $s$ bewirken.

2. Eine Abschätzung von $U^γ_D(s)$ durch $U^γ_D(s | π)$ für eine nicht-optimale Strategie π kann sehr fehlgeleitet sein.

   Stellen Sie sich einen Roboter vor, der den schnellsten Weg aus einem Hochhaus finden soll. Ist $\pi$ beispielsweise die Strategie, die vorschreibt in das jeweilige nächsthöhere Stockwerk zu gehen und im obersten Stockwerk den Fahrstuhl nach unten zu nehmen und dann das Gebäude zu verlassen, so hat der Zustand, bei dem sich der Agent im Erdgeschoss befindet, bzgl. $\pi$ einen sehr kleinen Nutzen (schließlich ist es für $\pi$ noch ein weiter Weg bis zum Ausgang). Für eine optimale Strategie $\pi^*$ hätte dieser Zustand allerdings einen sehr hohen Nutzen, da wir direkt zum Ausgang gehen könnten.


- Ein wichtiger Aspekt des aktiven Reinforcement Learning ist die **Exploration**, bei der in einem Zustand verschiedene Aktionen ausprobiert werden, um zu lernen, welche Aktion tatsächlich optimal ist.
- Sobald der Agent gelernt hat, welche Aktionen zu welchen Zuständen führen, kann dieses Wissen ausgenutzt werden (engl. "**exploitation**"), um gewinnbringend in der Umgebung zu agieren.

**Das Dilemma von Exploration vs. Exploitation** im Reinforcement Learning beschreibt, wie diese beiden Aspekte gegeneinander abgewogen werden müssen. Es stellt die Frage, wann ein Agent entscheiden kann, dass er genug ausprobiert hat und die bisher beste Strategie tatsächlich optimal ist.
- Eine anschauliche Darstellung dieses Dilemmas wird durch die Betrachtung des k-armigen Banditenproblems gegeben.

#### **Beispiel 1.** <a name="b1"></a> Das k-armige Banditenproblem

Ein <ins>einarmiger</ins> Bandit ist ein Spielautomat, der nach Einwerfen einer Münze einen Gewinn ausgibt, der gleichverteilt im Intervall $[a,b]$ wobei $a<b$ liegt. Ein <ins>k-armiger</ins> Bandit hat mehrere "Arme", die jeweils unterschiedliche Gewinnintervalle haben. Man wählt nach Einwerfen der Münze einen von den k Armen.
  
<ins>Betrachten wir den Fall k = 10</ins>: 
- wir nehmen an, dass wir eine große Menge an Geld zur Verfügung haben, z.B. 1000 EUR, D.h. wir könnten die ersten 100 Versuche nutzen, um jeden Arm jeweils 10 Mal zu betätigen und somit plausible Schätzungen der jeweiligen Gewinnintervalle $[a_i,b_i]$ zu erhalten.
- Anschließend könnten wir die verbleibenden 900 EUR nutzen, um ausschließlich den Arm zu betätigen, der am gewinnbringendsten erscheint (beispielsweise den Arm $i$, bei dem $a_i$ maximal ist).

**Die allgemeine Frage** beim k-Banditenproblem ist, wie viel Geld wir investieren sollten, um ein möglichst genaues Modell vom Banditen zu erhalten (Exploration), und wie viel Geld wir investieren sollten, um das gelernte Wissen zur Gewinnmaximierung zu nutzen (Exploitation).

So mopdelliert man das k-Banditenproblem:
- Es gibt einen *Startzustand* und *k Aktionen*, die aus dem Startzustand gewählt werden können.
- Nach Ausführen einer Aktion enden wir direkt in einem Zielzustand und erhalten den Gewinn.
- Es gibt also auch nur k verschiedene Strategien für dieses Problem.

In realen Anwendungsszenarien ist die Anzahl der möglichen Strategien allerdings zu groß und es ist nicht mäglich, zunächst alle mäglichen Strategien ausreichend auszuprobieren (wie im obigen Beispiel angedeutet) und dann die beste Strategie auszuwählen.

## 4.3.2. ε-greedy Learning <a name="4_3_2"></a>
---

- Die meisten Methoden des aktiven Reinforcement Learning verwenden eine *Meta-Strategie*, um das Dilemma zwischen Exploration und Exploitation zu adressieren. Diese Algorithmen beginnen mit einer initialen Strategie π und nutzen diese zunächst, um die Nutzenwerte der Zustände zu erlernen, die mit dieser Strategie erreicht werden.
- Je nach Meta-Strategie wird gelegentlich von der Strategie π abgewichen und in einem Zustand s eine andere Aktion a ≠ π(s) gewählt, um neue Teile des Zustandsraums zu erkunden und die Nutzenwerte der Zustände entsprechend zu aktualisieren.
- Wenn sich in den Nutzenwerten abzeichnet, dass in einem Zustand s tatsächlich eine andere Aktion als π(s) besser ist, wird π entsprechend geändert.

- Die einfachste Instanz einer solchen Meta-Strategie ist die **ε-greedy-Strategie**. Hierbei ist $ε \in [0,1]$ ein Parameter, der angibt, wie häufig die Exploration der Exploitation vorgezogen wird, bzw. der die Wahrscheinlichkeit bestimmt, mit der eine zufällige Aktion (Exploration) anstelle der besten bekannten Aktion (Exploitation) gewählt wird.
    - Wenn `ε = 0`, wird immer die beste bekannte Aktion (Exploitation) gewählt, es findet also keine Exploration statt.
    - Wenn `ε = 1`, wird immer eine zufällige Aktion gewählt, es findet also nur Exploration statt.
    - Mit Wahrscheinlichkeit `ε zwischen 0 und 1` wird in einem Zustand s eine Balance zwischen Exploration und Exploitation gefunden D.h. Es wird eine zufällige Aktion ausgewählt und mit Wahrscheinlichkeit 1 − ε wird π(s) ausgeführt. Beispielsweise bedeutet ε = 0.1, dass in 10% der Fälle eine zufällige Aktion (Exploration) und in 90% der Fälle die beste bekannte Aktion (Exploitation) gewählt wird.

Durch die Nutzung einer solchen Meta-Strategie können beide oben genannten Probleme gelöst werden:

1. Wenn wir hinreichend oft zufällige Aktionen in einem Zustand $s$ ausführen, können wir sowohl $P(s,a,s')$ als auch $R(s,a,s')$ für $a \neq \pi(s)$ approximieren.

2. Wenn die Strategie $\pi$ nicht optimal ist, können dennoch durch eine zufällig gute Auswahl von Aktionen die tatsächlichen Nutzen von Zuständen erkannt werden. Für das Beispiel des Hochhauses würden wir irgendwann zufällig im Erdgeschoss direkt die Aktion des Verlassens des Gebäudes ausführen und könnten damit direkt den Nutzen des Zustands im Erdgeschoss zu sein, erhöhen.Eine Kombination der **ε-greedy-Strategie mit der adaptiven dynamischen Programmierung** ([siehe Abschnitt 4.2.1](http://localhost:8888/lab/tree/KE4_Reinforcement_Learning/4.2.-Passives-Reinforcement-Learning.ipynb)) ist 
in Algorithmus 1 formalisiert.



![e-greedy](./e-greedy.PNG)

Der Algorithmus implementiert einen einzigen Schritt eines Agenten in einer Umgebung und erhält als Eingabe die aktuelle Beobachtung $o$ (falls der Agent im Startzustand ist, setzen wir $o = \text{null}$) und gibt die als nächstes auszuführende Aktion aus. Der Algorithmus besitzt eine Reihe von statischen Datenstrukturen, die über die verschiedenen Aufrufe des Algorithmus erhalten bleiben:

- Die Strategie $\pi$ ist die aktuell beste Strategie des Agenten.
- Der MDP $D$ ist das Modell der Umgebung, das der Agent mit jedem Aufruf besser erlernt.
- Die Funktion $u$ speichert die Nutzen der Zustände (bzgl. $\pi$).
- Die Funktionen $n_1$ und $n_2$ speichern die Häufigkeiten, wie oft in einem Zustand $s$ die Aktion $a$ ausgeführt wurde ($n_1(s,a)$) bzw. wie häufig beim Ausführen der Aktion $a$ in $s$ der Zustand $s'$ beobachtet wurde ($n_2(s,a,s')$).

Der Algorithmus funktioniert wie folgt:

1. Wenn `o` nicht `null` ist, werden die Häufigkeiten `n1(s,a)` und `n2(s,a,s')` aktualisiert.
2. Die Übergangswahrscheinlichkeiten `P(s,a,s'')` für alle `s''` in `S` werden aktualisiert.
3. Die Belohnung `R(s,a,s')` wird auf `r` gesetzt.
4. Die Nutzenfunktion `u` wird mittels der Funktion `VAL` aktualisiert (werden die Nutzen der Zustände bzgl. der aktuellen Strategie
aktualisier - Value Iteration.).
5. Die Strategie `π` wird mittels der Funktion `POL` aktualisiert (die Unterfunktion POL benutzt dazu direkt die Charakterisierung aus [Gleichung (1)](#g1)).
6. Wenn `s'` ein Terminalzustand ist, gibt der Algorithmus `null` zurück In diesem Fall muss der Agent in 
der Umgebung 
neu gestartet“ werden, wobei die statischen Datenstrukturen nicht neu initialisiert werden..
7. Mit Wahrscheinlichkeit `ε` gibt der Algorithmus eine zufällige Aktion `a` aus.
8. Mit Wahrscheinlichkeit `1 - ε` führt der Algorithmus die Aktion `π(s')` aus (oder `π(s0)` falls `o` `null` ist
Es kann wieder gezeigt werden, dass bei hinreichend vielen Aufrufen von Algorithmus 1 die Strategie π gegen 
die optimale Strategi  konvergiert).



#### **Beispiel 2.** <a name="b2"></a> ε-greedy Learning für das Staubsaugerproblem

Wir betrachten wieder das Beispiel des Staubsaugerroboters aus den vorherigen Unterkapiteln. Insbesondere ist die Menge der Zustände $S_{vc}$ gegeben durch

$$
S_{vc} = \{s_{1,1}^1, s_{1,1}^2, s_{0,1}^1, s_{0,1}^2, s_{1,0}^1, s_{1,0}^2, s_{0,0}^1, s_{0,0}^2, s_t\}
$$

wobei $s_{1,1}^1$ der Startzustand und $s_t$ der einzige Zielzustand ist. Die Menge der Aktionen $A_{vc}$ ist gegeben durch

$$
A_{vc} = \{\text{move}, \text{clean}, \text{charge}\}
$$

Nehmen wir weiterhin an, die initiale Strategie $\pi$ des Roboters ist gegeben durch

$$\pi(s_{1,1}^1) = \pi(s_{1,1}^2) = \pi(s_{0,1}^1) = \pi(s_{0,1}^2) = \pi(s_{1,0}^1) = \pi(s_{1,0}^2) = \pi(s_{0,0}^1) = \pi(s_{0,0}^2) = \text{move}$$

Wir führen Algorithmus 1 exemplarisch zweimal aus:

`Aufruf 1`

1. Beim ersten Aufruf befindet sich der Agent im Startzustand $s^1_{1,1}$. Da $o = null$ ist werden Zeilen 1–5 hierbei nicht ausgeführt.
2. Da noch keine Belohnungen beobachtet wurden, gilt initial $R(s,a,s') = 0$ für alle $s,s' \in S$, $a \in A$.
3. Es folgt, dass in **Zeile 6**, $u(s) = 0$ für alle $s \in S$ errechnet wird.
4. In **Zeile 7** ergibt sich, dass alle Strategien gleich gut/schlecht sind, wir ändern die aktuelle Strategie also nicht ab.
5. Da wir uns nicht in einem Zielzustand befinden und wir annehmen, dass der Zufallstest in Zeilen 9/10 zugunsten von Zeile 10 ausgefallen ist, geben wir $\pi(s^1_{1,1}) = \text{move}$ zurück.



`Aufruf 2`
1. Beim zweiten Aufruf erhält der Algorithmus die Beobachtung $$o = (s^1_{1,1}, \text{move}, s^2_{1,1}, -1)$$Mit anderen Worten, der Roboter ist erfolgreich in Raum r2 gewechselt und $o \neq null$. **Zeile 1** liefert den Wert `True`.

2. Wir aktualisieren in den **Zeilen 2–5** die Werte unserer Datenstrukturen wie folgt:
    * $n1(s^1_{1,1}, \text{move}) := n1(s^1_{1,1}, \text{move})+1 = 0+1 = 1$
    * $n2(s^1_{1,1}, \text{move}, s^2_{1,1}) := n2(s^1_{1,1}, \text{move}, s^2_{1,1})+1 = 0+1 = 1$
    * Die Aktualisierung von $P(s,a,s'')$ erfolgt gemäß der Formel: $$P(s,a,s'') := \frac{n2(s,a,s'')}{n1(s,a)}$$Daher ist $$P(s^1_{1,1}, \text{move}, s^2_{1,1}) := \frac{1}{1} = 1$$
    * $P(s^1_{1,1}, \text{move}, s'') := 0$ für alle $s'' \neq s^2_{1,1}$
    * $R(s^1_{1,1}, \text{move}, s^2_{1,1}) := -1$

3. In **Zeile 6** ergibt sich $$u(s) := 0 \qquad \qquad \forall \;s$$
   Insbesondere ändert sich also (zunächst) nicht der Wert von $u(s^1_{1,1})$ und bleibt 0, da er in Zeile 6 über das Maximum aller Aktionen berechnet wird und nur der Nutzen bei Ausführen der Aktion `move` auf -1 gesetzt wurde.
5. In **Zeile 7** ergibt sich, dass für den Zustand $s^1_{1,1}$ jede Aktion außer move einen maximal erwarteten Nutzen von 0 bringt. Wir setzen zufällig $π(s^1_{1,1})$ = clean, für alle anderen Zustände behalten wir die Strategie.
6. In **Zeilen 9/10** wählen wir eine zufällige Aktion `charge` anstelle von $π(s^1_{1,1})$.


`Aufruf 3`

Angenommen, der Roboter erhält beim dritten Aufruf die Beobachtung: $o = (s_{1,1}^2, \text{clean}, s_{1,0}^2, 10)$f 3`

1. Da $o \neq null$, führen wir die Zeilen 1–5 aus:
    * $n1(s^2_{1,1}, \text{clean}) := n1(s^2_{1,1}, \text{clean})+1 = 0+1 = 1$
    * $n2(s^2_{1,1}, \text{clean}, s^2_{1,0}) := n2(s^2_{1,1}, \text{clean}, s^2_{1,0})+1 = 0+1 = 1$
    * Die Aktualisierung von $P(s,a,s'')$ erfolgt gemäß der Formel: $$P(s,a,s'') := \frac{n2(s,a,s'')}{n1(s,a)}$$Daher ist $$P(s^2_{1,1}, \text{clean}, s^2_{1,0}) := \frac{1}{1} = 1$$
    * $P(s^2_{1,1}, \text{clean}, s'') := 0$ für alle $s'' \neq s^2_{1,0}$
    * $R(s^2_{1,1}, \text{clean}, s^2_{1,0}) := 10$

2. In **Zeile 6** ergibt sich $$u(s) := 0 \qquad \qquad \forall \;s$$
   Insbesondere ändert sich also (zunächst) nicht der Wert von $u(s^2_{1,1})$ und bleibt 0, da er in Zeile 6 über das Maximum aller Aktionen berechnet wird und nur der Nutzen bei Ausführen der Aktion `clean` auf 10 gesetzt wurde.
3. In **Zeile 7** ergibt sich, dass für den Zustand $s^2_{1,1}$ jede Aktion außer clean einen maximal erwarteten Nutzen von 0 bringt. Wir setzen zufällig $π(s^2_{1,1})$ = move, für alle anderen Zustände behalten wir die Strategie.
4. In **Zeilen 9/10** wählen wir eine zufällige Aktion `charge` anstelle von $π(s^2_{1,1})$.

Dies zeigt, wie der Algorithmus die Belohnungen und Strafen berücksichtigt, die er erhält, um seine Strategie anzupassen und effektiv in seiner Umgebung zu navigieren. 😊 aktualisiert wurden.

Ein Nachteil von Algorithmus 1 ist, dass in Zeile 6 die Nutzen aller Zustände (potenziell) neu berechnet werden und anschließend die Strategie aktualisiert wird. Dies ist begründet in der Nutzung der adaptiven dynamischen Programmierung, bei der wir die gleiche Kritik schon in Abschnitt 4.2.2 benutzt haben, um den Ansatz des Temporal Difference Learning (TD) zu motivieren. In gleicher Weise können wir allerdings auch den TD-Ansatz für das aktive Reinforcement Learning erweitern. Dies werden wir im nächsten Abschnitt tun.


## 4.3.3. Q-Learning <a name="4_3_3"></a>
---

- Q-Learning ist ein Temporal Difference (TD) Ansatz für aktives Reinforcement Learning. Es unterliegt den meisten modernen Ansätzen zum Reinforcement Learning.
- Im Gegensatz zu anderen Ansätzen <ins>lernt Q-Learning nicht die Nutzen der Zustände direkt, sondern die sogenannte **Q-Funktion**</ins>.
- Die Q-Funktion $Q_\gamma$ weist jedem Zustand $s$ und jeder Aktion $a$ den erwarteten maximalen Nutzen zu, den man bei Verfolgung der optimalen Strategie nach Ausführung der Aktion $a$ in $s$ erhält.
- Der Zusammenhang zwischen $Q_\gamma$ und den Nutzen der Zustände ist gegeben durch $$U_\gamma D(s) = \max_{a \in A} Q_\gamma (a,s)$$ für alle Zustände $s$.
- Wenn die wahren $Q_\gamma$-Werte eines jeden Zustands gegeben sind, kann die optimale Strategie $\pi^*$ leicht bestimmt werden durch $$\pi^*(s) = \arg\max_{a \in A} Q_\gamma (a,s) \tag{2}$$
- Der **Vorteil** der Nutzung der $Q_\gamma$-Funktion anstelle der Nutzenwerte ist, <ins>dass wir zur Bestimmung der optimalen Strategie kein Modell der Umgebung (insbesondere der Übergangswahrscheinlichkeiten $P$) erlernen müssen</ins>.
- Aus diesem Grund nennt man das Q-Learning auch einen **modellfreien Ansatz** (engl. model-free method).

![q-learning-with-e-greedy](./q-learning-with-e-greedy.PNG)

- Um die Q-Funktion zu erlernen, verwendet Q-Learning den Temporal Difference (TD) Ansatz, .
- Bei jeder Beobachtung $o = (s,a,s',r)$ wird die aktuelle Schätzung $q_\gamma (a,s)$ aktualisiert.
- Die Aktualisierung erfolgt durch eine Updateregel, die für das Q-Learning definiert ist durch: $$q_\gamma (a,s) := q_\gamma (a,s)+\alpha(r +\gamma \max_{a' \in A} q_\gamma (a',s')−q_\gamma (a,s))$$
- Hierbei ist $\alpha \in [0,1]$ der **Lernparameter** und $r +\gamma \max_{a' \in A} q_\gamma (a',s')$ die **Schätzung des Q-Wertes** aus der gerade getätigten Beobachtung.
- In der Praxis ist $\alpha$ nicht konstant, sondern wird mit steigender Anzahl von Beobachtungen geringer.
- Um das Exploration vs. Exploitation-Dilemma mit Q-Learning zu lösen, muss der oben beschriebene Ansatz mit einer Meta-Strategie zur Exploration kombiniert werden.
- Eine Kombination der ε-greedy-Strategie mit Q-Learning ist in Algorithmus 2 formalisiert.
- Unter wenigen formalen Annahmen kann gezeigt werden, dass bei hinreichend vielen Aufrufen von Algorithmus 2 die Strategie $\pi$ gegen die optimale Strategie konvergiert.
- Im direkten Vergleich von Algorithmus 1 und Algorithmus 2 ist Q-Learning einfacher und benötigt weniger Speicherplatz.
- Ein **Nachteil** des Q-Learnings gegenüber des Ansatzes der adaptiven dynamischen Programmierung ist jedoch, dass ersteres weitaus <ins>langsamer</ins> gegen die optimale Strategie konvergiert.

#### **Beispiel 3.** <a name="b3"></a> Q-Learning für das Staubsaugerproblem


* Wir führen Beispiel 2 fort. Wir nehmen dabei an, dass $\gamma = 0.9$, $\alpha = 0.1$ und die initiale Strategie $\pi$ des Roboters gegeben ist durch

$$
\pi(s_{1,1}^1) = \pi(s_{1,1}^2) = \pi(s_{0,1}^1) = \pi(s_{0,1}^2) = \pi(s_{1,0}^1) = \pi(s_{1,0}^2) = \pi(s_{0,0}^1) = \pi(s_{0,0}^2) = \text{move}
$$

Wir führen Algorithmus 2 exemplarisch zweimal aus:

1. Beim ersten Aufruf befindet sich der Agent im Startzustand $s_{1,1}^1$. Zeilen 1–3 werden hierbei nicht ausgeführt. Da wir uns nicht in einem Zielzustand befinden und wir annehmen, dass der Zufallstest in den Zeilen 5/6 zugunsten von Zeile 6 ausgefallen ist, geben wir $\pi(s_{1,1}^1) = \text{move}$ zurück.

2. Beim zweiten Aufruf erhält der Algorithmus die Beobachtung $o = (s_{1,1}^1, \text{move}, s_{1,1}^2, -1)$. Mit anderen Worten, der Roboter ist erfolgreich in Raum r2 gewechselt. Wir aktualisieren in den Zeilen 2 und 3 die Werte $q_\gamma (\text{move}, s_{1,1}^1)$ und $\pi(s_{1,1}^1)$ wie folgt:

$$
q_\gamma (\text{move}, s_{1,1}^1) := q_\gamma (\text{move}, s_{1,1}^1) + \alpha(r + \gamma \max\{q_\gamma (\text{move}, s_{1,1}^2), q_\gamma (\text{clean}, s_{1,1}^2), q_\gamma (\text{charge}, s_{1,1}^2)\} - q_\gamma (\text{move}, s_{1,1}^1))
$$

$$
= 0 + 0.1(-1 + 0.9 \cdot 0 - 0) = -0.1
$$

und

$$
\pi(s_{1,1}^1) = \text{clean}
$$

Beachten Sie, dass für Bestimmung von $\pi(s_{1,1}^1)$ oben gilt

$$
q_\gamma (\text{move}, s_{1,1}^1) = -0.1, \quad q_\gamma (\text{clean}, s_{1,1}^1) = 0, \quad q_\gamma (\text{charge}, s_{1,1}^1) = 0
$$

Die Wahl $\pi(s_{1,1}^1) = \text{clean}$ wurde also zufällig aus den beiden maximalen Werten für clean und charge getroffen. In den Zeilen 9/10 wählen wir eine zufällige Aktion charge anstelle von $\pi(s_{1,1}^2)$.

## 4.3.4. Unterschiede zwischen $\epsilon$-greedy-Learning und Q-Learning <a name="4_3_4"></a>
---

Sicher, hier ist eine Tabelle, die die Unterschiede zwischen Epsilon-Greedy-Learning und Q-Learning zusammenfasst:

|  | Epsilon-Greedy-Learning | Q-Learning |
|---|---|---|
| **Was es ist** | Eine Methode zur Aktionenauswahl | Ein spezifischer Algorithmus im Reinforcement Learning |
| **Hauptfunktion** | Balanciert Exploration und Exploitation | Schätzt den erwarteten Nutzen einer Aktion in einem bestimmten Zustand |
| **Wie es funktioniert** | Wählt mit einer Wahrscheinlichkeit von ε eine zufällige Aktion (Exploration) und mit einer Wahrscheinlichkeit von 1-ε die Aktion mit dem höchsten erwarteten Nutzen (Exploitation) | Lernt die Q-Werte für jedes Zustands-Aktions-Paar und verwendet diese, um die optimale Strategie zu bestimmen |
| **Zusätzliche Informationen** | Wird oft als Teil des Q-Learning-Prozesses verwendet | Ist ein wertbasierter, modellfreier und off-policy Algorithmus |--|
| **Verwendung während des Tests** | Epsilon-Greedy wird oft auch während der Testphase verwendet, da im Reinforcement Learning im Gegensatz zum überwachten Lernen kein unabhängiger Testdatensatz vorhanden ist³. | Q-Learning wird hauptsächlich während der Trainingsphase verwendet, um die Q-Werte zu lernen und die optimale Strategie zu bestimmen¹. |
| **Abhängigkeit von der Policy** | Epsilon-Greedy ist eine Methode zur Aktionenauswahl und ist nicht direkt von der Policy abhängig¹. | Q-Learning ist ein off-policy Algorithmus. Es schätzt die Belohnung für Zustands-Aktions-Paare basierend auf der optimalen (greedy) Policy, unabhängig von den Aktionen des Agenten¹. |
| **Erfordernis eines Modells** | Epsilon-Greedy erfordert kein Modell der Umgebung¹. | Q-Learning ist ein modellfreier Ansatz. Der Agent erforscht die Umgebung und lernt aus den Ergebnissen der Aktionen direkt, ohne ein internes Modell oder einen Markov-Entscheidungsprozess zu konstruieren¹. |
| **Anpassungsfähigkeit** | Der Epsilon-Wert in Epsilon-Greedy kann im Laufe der Zeit angepasst werden, um das Gleichgewicht zwischen Exploration und Exploitation zu optimieren⁴. | Q-Learning passt die Q-Werte basierend auf den erhaltenen Belohnungen an, um die optimale Strategie zuleration-function.

> **Definition 2. Multimenge der Probeläufen und der Nutzen $U^{\gamma}_{\partial}(s | \pi)$**
>
> 


#### **Beispiel 1.** <a name="b1"></a> Das k-armige Banditenproblem

Ein

---
<a name="up"></a>**`[Go Up!^](#up)**

##### Sources:

(1) Why does Q-Learning use epsilon-greedy during testing?. https://stats.stackexchange.com/questions/270618/why-does-q-learning-use-epsilon-greedy-during-testing.
(2) Epsilon-Greedy Q-learning | Baeldung on Computer Science. https://www.baeldung.com/cs/epsilon-greedy-q-learning.
(3) Epsilon and learning rate decay in epsilon greedy q learning. https://stackoverflow.com/questions/53198503/epsilon-and-learning-rate-decay-in-epsilon-greedy-q-learning.
(4) Exploration in Q learning: Epsilon greedy vs Exploration function. https://datascience.stackexchange.com/questions/94029/exploration-in-q-learning-epsilon-greedy-vs-exploration-function.